<a href="https://colab.research.google.com/github/shun0109/colab/blob/main/processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install seaborn --quiet

import pandas as pd
import numpy as np
from scipy import stats, integrate
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict
from sklearn.linear_model import LogisticRegression, LinearRegression,Ridge,SGDClassifier
from sklearn.metrics import accuracy_score,roc_curve, confusion_matrix,classification_report,mean_squared_error,precision_score, \
recall_score,f1_score,classification_report,precision_recall_curve, roc_auc_score, ConfusionMatrixDisplay
from collections import Counter
from IPython.core.display import display, HTML
import glob
%matplotlib inline

In [10]:
# !cd

In [9]:
pd.read_csv('/01_Hokkaido_20201_20251.csv', encoding="CP932")

,種類,価格情報区分,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,取引価格（総額）,...,用途,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時期,取引の事情等
0,宅地(土地),不動産取引価格情報,住宅地,1101,北海道,札幌市中央区,旭ケ丘,円山公園,30分～60分,94000000,...,NaN,住宅,北西,市道,8.2,１種住居,60.0,200.0,2024年第3四半期,NaN
1,宅地(土地),不動産取引価格情報,住宅地,1101,北海道,札幌市中央区,旭ケ丘,円山公園,23,25000000,...,NaN,事務所,東,道道,15.0,１種住居,60.0,200.0,2023年第4四半期,NaN
2,宅地(土地),不動産取引価格情報,住宅地,1101,北海道,札幌市中央区,旭ケ丘,円山公園,23,16000000,...,NaN,住宅,西,市道,6.0,１低住専,40.0,80.0,2023年第4四半期,NaN
3,宅地(土地),不動産取引価格情報,住宅地,1101,北海道,札幌市中央区,旭ケ丘,円山公園,21,92000000,...,NaN,住宅,西,市道,14.5,１種住居,60.0,200.0,2023年第2四半期,NaN
4,宅地(土地と建物),不動産取引価格情報,住宅地,1101,北海道,札幌市中央区,旭ケ丘,円山公園,28,78000000,...,住宅,住宅,北東,道道,15.0,１種住居,60.0,200.0,2023年第1四半期,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68841,宅地(土地と建物),不動産取引価格情報,住宅地,1694,北海道,目梨郡羅臼町,（大字なし）,NaN,NaN,690000,...,住宅、倉庫,住宅,北西,道道,16.0,都計外,NaN,NaN,2020年第4四半期,NaN
68842,宅地(土地),不動産取引価格情報,住宅地,1694,北海道,目梨郡羅臼町,（大字なし）,NaN,NaN,600000,...,NaN,NaN,南東,国道,25.0,都計外,NaN,NaN,2020年第3四半期,NaN
68843,宅地(土地と建物),不動産取引価格情報,住宅地,1694,北海道,目梨郡羅臼町,（大字なし）,NaN,NaN,3200000,...,住宅,住宅,接面道路無,NaN,NaN,都計外,NaN,NaN,2020年第2四半期,NaN
68844,宅地(土地),不動産取引価格情報,住宅地,1694,北海道,目梨郡羅臼町,（大字なし）,NaN,NaN,3000000,...,NaN,倉庫,北西,道道,14.0,都計外,NaN,NaN,2020年第1四半期,NaN


In [14]:
file_paths = glob.glob('/*.csv')

dataframes = []
# columns = ['Type', 'Region', 'Prefecture','MinTimeToNearestStation',
#        'MaxTimeToNearestStation', 'TradePrice', 'Area',
#        'AreaIsGreaterFlag',
#        'Frontage', 'FrontageIsGreaterFlag', 'TotalFloorArea',
#        'TotalFloorAreaIsGreaterFlag', 'BuildingYear',
#        'Structure', 'Purpose', 'Direction', 'Classification', 'Breadth',
#        'CityPlanning', 'CoverageRatio', 'FloorAreaRatio', 'Year', 'Quarter']
for file_path in file_paths:
    temp = pd.read_csv(file_path, low_memory=False,encoding="CP932")
    dataframes.append(temp)

# Concatenate all the DataFrames into one big DataFrame
df = pd.concat(dataframes, ignore_index=True)
df

,種類,価格情報区分,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,取引価格（総額）,...,用途,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時期,取引の事情等
0,宅地(土地と建物),成約価格情報,NaN,43101,熊本県,熊本市中央区,出水,NaN,NaN,31000000,...,NaN,NaN,西,NaN,5.0,２中住専,60.0,200.0,2021年第1四半期,NaN
1,宅地(土地と建物),成約価格情報,NaN,43101,熊本県,熊本市中央区,出水,水前寺公園,5,57000000,...,NaN,NaN,北西,私道,4.0,２中住専,60.0,200.0,2021年第1四半期,NaN
2,宅地(土地と建物),成約価格情報,NaN,43101,熊本県,熊本市中央区,本荘,南熊本,20,45000000,...,NaN,NaN,西,公道,4.0,２種住居,60.0,200.0,2021年第2四半期,NaN
3,宅地(土地と建物),成約価格情報,NaN,43101,熊本県,熊本市中央区,出水,水前寺公園,20,39000000,...,NaN,NaN,NaN,NaN,NaN,１低住専,40.0,80.0,2021年第3四半期,NaN
4,宅地(土地と建物),成約価格情報,NaN,43101,熊本県,熊本市中央区,本荘,河原町(熊本),5,78000000,...,NaN,NaN,南東,公道,5.0,２種住居,60.0,200.0,2021年第3四半期,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1288084,宅地(土地と建物),不動産取引価格情報,住宅地,44462,大分県,玖珠郡玖珠町,大字山田,豊後森,23,16000000,...,住宅,住宅,南,町道,5.0,１中住専,60.0,200.0,2021年第4四半期,NaN
1288085,宅地(土地と建物),不動産取引価格情報,住宅地,44462,大分県,玖珠郡玖珠町,大字山田,豊後森,26,20000000,...,倉庫、店舗,事務所,北,国道,15.0,非線引き,NaN,NaN,2020年第1四半期,NaN
1288086,宅地(土地と建物),不動産取引価格情報,住宅地,44462,大分県,玖珠郡玖珠町,大字四日市,豊後森,28,200000,...,住宅,倉庫,南,県道,NaN,非線引き,NaN,NaN,2023年第4四半期,NaN
1288087,宅地(土地),不動産取引価格情報,住宅地,44462,大分県,玖珠郡玖珠町,大字四日市,豊後森,18,4800000,...,NaN,NaN,南,県道,15.0,非線引き,NaN,NaN,2023年第3四半期,NaN


In [16]:
df.columns

Index(['種類', '価格情報区分', '地域', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称',
       '最寄駅：距離（分）', '取引価格（総額）', '坪単価', '面積（㎡）', '取引価格（㎡単価）', '土地の形状', '間口',
       '延床面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '前面道路：方位', '前面道路：種類',
       '前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）', '容積率（％）', '取引時期', '取引の事情等'],
      dtype='object')